- 模型应用
  - 模型加载
  - 模型测试集评估
    - 准确率
    - 召回率
    - F1


In [ ]:
# 随机评论数据测试

import config
import model
import dataset
import jieba
import numpy
import torch

model = model.SentimentAnalysisModel().load('/workspace/project/deep_learning_2024/pt/sentiment_analysis_GRU_512_0.2_ReLU_50_100_10_2_128_50_0.005_0.1.pt').to(config.DEVICE)
dataset = dataset.SentimentAnalysisDataset(stopwords_path=config.STOPWORDS_PATH,
                                           train_data_path=config.TRAIN_DATA_PATH,
                                           validation_data_path=config.VALIDATION_DATA_PATH,
                                           test_data_path=config.TEST_DATA_PATH,
                                           sequence_length=config.SEQUENCE_LENGTH,
                                           word2vec_vector_size=config.WORD2VEC_VECTOR_SIZE,
                                           word2vec_epochs=config.WORD2VEC_EPOCHS,
                                           word2vec_min_count=config.WORD2VEC_MIN_COUNT,
                                           )
# dataset.label2emotion
# label2emotion = {
#     0: '非常满意',
#     1: '满意',
#     2: '中立',
#     3: '不满意',
#     4: '非常不满意',
#     5: '无效评论'
# }

contents = ["感觉这个秤非常小巧精致实用。自从看到家人买来这个秤以后，其他的包括秤（包括金器秤）都被闲置了",
            "第三次购买了，还算可以，希望能更耐用一些。",
            "没什想说的，中规中矩，没什么色差",
            "秤盘不平，一边高一边低。这个价位不应该这样了。按键硬，按起来费力。底座盘不平，有一个脚悬空",
            "底座四个点不平，质量不行，本来就是有精度要求，结果买一个底座不平的电子秤。简直是一个笑话",
            "182712798172498172948718924791872489172894817"]

for content in contents:

    # 1
    content = content.strip()
    # content = self.__clean_redundant(content)
    # jieba分词
    seg_list = jieba.cut(content, cut_all=False)
    token = []
    for seg in seg_list:
        # 去除停用词
        # if seg not in dataset.stopwords:
        token.append(seg)

    train_tokens_truncated = token[:config.SEQUENCE_LENGTH]

    # self.train_vecs = [self.__token2vecs_and_padding(token, sequence_length) for token in train_tokens_truncated]

    vecs = []
    for seg in train_tokens_truncated:
        if seg in dataset.word2vec.wv:
            vecs.append(dataset.word2vec.wv[seg])
    if config.SEQUENCE_LENGTH > len(vecs):
        vecs += [numpy.zeros(dataset.word2vec.wv.vector_size)] * (config.SEQUENCE_LENGTH - len(vecs))
        # return vecs

    data = torch.Tensor(numpy.array(vecs)).unsqueeze(0).to(config.DEVICE)
    output, _ = model(data)
    predicted = output.argmax(dim=1).cpu().numpy()
    print(content, predicted[0], dataset.label2emotion.get(predicted[0]))

In [ ]:
# 随机评论数据测试

import config
import model
import dataset
import numpy
import torch
import word_embedding

model = model.SentimentAnalysisModel().load('pt/sentiment_analysis_GRU_512_0.2_ReLU_50_100_10_2_128_50_0.005_0.1.pt').to(config.DEVICE)
word_embedding = word_embedding.WordEmbedding().load('pt/word_embedding_GRU_512_0.2_ReLU_50_100_10_2_128_50_0.005_0.1.pt').to(config.DEVICE)

# test_loader = dataset.SentimentAnalysisDataset(data_path=config.TEST_DATA_PATH, word_embedding=word_embedding,
#                                                sequence_length=config.SEQUENCE_LENGTH, ).construct_dataloader(
#     batch_size=config.TRAIN_BATCH_SIZE)

contents = ["感觉这个秤非常小巧精致实用。自从看到家人买来这个秤以后，其他的包括秤（包括金器秤）都被闲置了",
            "第三次购买了，还算可以，希望能更耐用一些。",
            "没什想说的，中规中矩，没什么色差",
            "秤盘不平，一边高一边低。这个价位不应该这样了。按键硬，按起来费力。底座盘不平，有一个脚悬空",
            "底座四个点不平，质量不行，本来就是有精度要求，结果买一个底座不平的电子秤。简直是一个笑话",
            "182712798172498172948718924791872489172894817"]

vecs = word_embedding.sentences2vecs(contents)
for vec in vecs:
    vec_padding = dataset.vec_padding(vec, config.SEQUENCE_LENGTH,word_embedding.word2vec_vector_size)
    data = torch.Tensor(numpy.array(vec_padding)).unsqueeze(0).to(config.DEVICE)
    output, _ = model(data)
    predicted = output.argmax(dim=1).cpu().numpy()
    print(content, predicted[0], dataset.label2emotion.get(predicted[0]))


